# Setting up enviorment

In [1]:
!pip install -q transformers datasets accelerate evaluate scikit-learn torch
!pip install -U transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import random
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

2025-11-13 05:56:42.106187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763013402.524935      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763013402.628077      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os, random, numpy as np, pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import inspect
from transformers import TrainingArguments

# Import file and analyze it 

In [5]:
Seed=42
random.seed(Seed)
np.random.seed(Seed)

In [6]:
df = pd.read_csv('/kaggle/input/sensitive-analysis/sentiment-analysis.csv', encoding='utf-8-sig')
print("Columns:", df.columns.tolist())
print(df.head(5))

Columns: ['Text', ' Sentiment', ' Source', ' Date/Time', ' User ID', ' Location', ' Confidence Score']
                                               Text  Sentiment         Source  \
0                              I love this product!   Positive        Twitter   
1                         The service was terrible.   Negative   Yelp Reviews   
2                            This movie is amazing!   Positive           IMDb   
3  I'm so disappointed with their customer support.   Negative   Online Forum   
4                Just had the best meal of my life!   Positive    TripAdvisor   

              Date/Time       User ID      Location   Confidence Score  
0   2023-06-15 09:23:14      @user123      New York               0.85  
1   2023-06-15 11:45:32       user456   Los Angeles               0.65  
2   2023-06-15 14:10:22   moviefan789        London               0.92  
3   2023-06-15 17:35:11    forumuser1       Toronto               0.78  
4   2023-06-16 08:50:59      foodie22        

In [16]:

df[label_col] = df[label_col].str.strip().str.lower()

# Create label mappings
unique = sorted(df[label_col].unique())
label2id = {lab: i for i, lab in enumerate(unique)}
id2label = {i: lab for lab, i in label2id.items()}

# Add numeric label column
df["label"] = df[label_col].map(label2id)

# Show mapping and sample rows
print("Label map:", label2id)
print(df[[text_col, label_col, "label"]].head())


Label map: {'negative': 0, 'positive': 1}
                                               Text Sentiment  label
0                              I love this product!  positive      1
1                         The service was terrible.  negative      0
2                            This movie is amazing!  positive      1
3  I'm so disappointed with their customer support.  negative      0
4                Just had the best meal of my life!  positive      1


# Preprocessing
### we may observe our sentiments are positive or negative convert them to digit 0 or 1

# Lets convert our data set into Hugging face format 
### for testing we are using small dataset

In [17]:
data=Dataset.from_pandas(df[[text_col,"label"]])
data_split=data.train_test_split(test_size=0.2,seed=Seed)
dataset=DatasetDict({"train":data_split["train"],"validation":data_split["test"]})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Text', 'label'],
        num_rows: 364
    })
    validation: Dataset({
        features: ['Text', 'label'],
        num_rows: 92
    })
})


# Importing our BERT Model

In [18]:
model_base = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_base)

def preprocess(batch):
    return tokenizer(batch[text_col], padding="max_length", max_length=128, truncation=True)

dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/364 [00:00<?, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

In [19]:
num_labels=len(label2id)
model=AutoModelForSequenceClassification.from_pretrained(model_base,num_labels=num_labels,id2label=id2label,label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# evaulation metrices
from inspect import signature

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

In [24]:
# hugging face default parameters for model training 
training_args = TrainingArguments(
    output_dir="./bert_sentiment",
    learning_rate=2e-5,
    eval_strategy="epoch", 
    save_strategy="epoch",
    
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_39/1875103790.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
import transformers
print(transformers.__version__)


4.57.1


In [26]:
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.000400,0.000144,1.000000,1.000000
2,0.000100,0.000066,1.000000,1.000000
3,0.000100,0.000048,1.000000,1.000000
4,0.000100,0.000041,1.000000,1.000000
5,0.000100,0.000039,1.000000,1.000000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=115, training_loss=0.00023278909755627747, metrics={'train_runtime': 47.9871, 'train_samples_per_second': 37.927, 'train_steps_per_second': 2.396, 'total_flos': 119715530188800.0, 'train_loss': 0.00023278909755627747, 'epoch': 5.0})

In [27]:
pred_out = trainer.predict(dataset["validation"])
preds = np.argmax(pred_out.predictions, axis=-1)
labels = pred_out.label_ids

print("Accuracy:", accuracy_score(labels, preds))
print("Macro F1:", f1_score(labels, preds, average="macro"))
label_names = [id2label[i] for i in sorted(set(labels))] 
print("\nClassification Report:\n", classification_report(labels, preds, target_names=label_names))
print("\nConfusion Matrix:\n", confusion_matrix(labels, preds))


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Accuracy: 1.0
Macro F1: 1.0

Classification Report:
               precision    recall  f1-score   support

    negative       1.00      1.00      1.00        39
    positive       1.00      1.00      1.00        53

    accuracy                           1.00        92
   macro avg       1.00      1.00      1.00        92
weighted avg       1.00      1.00      1.00        92


Confusion Matrix:
 [[39  0]
 [ 0 53]]


In [28]:
trainer.save_model("./bert_sentiment/final")
tokenizer.save_pretrained("./bert_sentiment/final")
print("Saved to ./bert_sentiment/final")


Saved to ./bert_sentiment/final


In [29]:
import torch

test_texts = [
    "I thought it would be terrible but it exceeded my expectations!",
    "The product isn't bad but the customer service ruined everything.",
    "Not the best, not the worst, just okay I guess.",
    "I wanted to love it so badly but it let me down completely.",
    "Despite the flaws, I'm surprisingly happy with my purchase.",
    "The reviews said it was amazing but I found it utterly disappointing.",
    "It's fine if you have low expectations, otherwise you'll hate it.",
    "I can't believe how much I regret buying this piece of junk.",
    "Mixed feelings - great price but questionable quality.",
    "Honestly expected nothing and still managed to be impressed!",
    "Would have been perfect if not for that one major issue.",
    "I'm not sure if I like it or not, very confusing experience.",
    "Started great but ended up being a waste of money.",
    "The worst part? It actually works but I still hate using it.",
    "Surprisingly decent for the price, can't complain much."
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_inputs = tokenizer(test_texts, padding="max_length", max_length=128, truncation=True, return_tensors="pt")
test_inputs = {k: v.to(device) for k, v in test_inputs.items()}

model.eval()
with torch.no_grad():
    outputs = model(**test_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

results = pd.DataFrame({
    "Text": test_texts,
    "Predicted_Label": [id2label[pred.item()] for pred in predictions],
    "Confidence": torch.softmax(outputs.logits, dim=-1).max(dim=-1).values.cpu().numpy()
})

print(results.to_string(index=False))

                                                                 Text Predicted_Label  Confidence
      I thought it would be terrible but it exceeded my expectations!        negative    0.999765
    The product isn't bad but the customer service ruined everything.        negative    0.999848
                      Not the best, not the worst, just okay I guess.        positive    0.949247
          I wanted to love it so badly but it let me down completely.        negative    0.993570
          Despite the flaws, I'm surprisingly happy with my purchase.        positive    0.999775
The reviews said it was amazing but I found it utterly disappointing.        negative    0.999819
    It's fine if you have low expectations, otherwise you'll hate it.        negative    0.999820
         I can't believe how much I regret buying this piece of junk.        negative    0.999788
               Mixed feelings - great price but questionable quality.        negative    0.999864
         Honestly ex

In [30]:
from google.colab import files
import shutil
shutil.make_archive('bert_sentiment_model', 'zip', './bert_sentiment/final')

'/kaggle/working/bert_sentiment_model.zip'